In [1]:
import pandas as pd
import json
from geopy.geocoders import Nominatim

rentdata_df = pd.read_json('rent_data.json')

In [2]:
rentdata_df

,title,price,area,address,description,type,type_estate,law,floor,bedroom,toilet,direction,image_links,phonenumber
0,"Cho thuê căn hộ tầng 3, 4, 5 trong toà nhà 8 t...",8.700.000 đ/tháng,65 m²,"Đường Giang Văn Minh, Phường Đội Cấn, Quận Ba ...","Nhà mình còn 3 căn hộ tầng 3, 4, 5 trong toà n...",Cho Thuê,Chung cư,NaN,NaN,1 phòng,1 WC,None,https://cloud.muaban.net/images/thumb-detail/2...,913236393
1,Cho thuê mặt bằng tầng trệt đường số 6 khu Cit...,16.000.000 đ/tháng,100 m²,"Cityland Park Hills, Đường số 6, Phường 10, Qu...",Cho thuê mặt bằng tầng trệt đường số 6 khu Cit...,Cho Thuê,Mặt bằng kinh doanh,NaN,NaN,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,903101485
2,CHO THUÊ VP GIÁ RẺ NHẤT BÌNH THẠNH,12.000.000 đ/tháng,56 m²,"110, Đường Đinh Bộ Lĩnh, Phường 26, Quận Bình ...",Cơ hội vàng cho doanh nghiệp: Văn phòng đẳng c...,Cho Thuê,Văn phòng,NaN,NaN,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,983414444
3,"Cho thuê cửa hàng phố Lê Duẩn, nhìn sang Điện ...",123.456 đ/tháng,60 m²,"Đường Lê Duẩn, Quận Ba Đình, Hà Nội","Cần cho thuê cửa hàng phố Lê Duẩn, nhìn sang Đ...",Cho Thuê,"Cửa hàng, shophouse",NaN,NaN,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,934188668
4,"Chính chủ cho Thuê Nhà mặt phố Kim Đồng, Phườn...",30.000.000 đ/tháng,60 m²,"Đường Kim Đồng, Phường Giáp Bát, Quận Hoàng Ma...","Cho Thuê Nhà mặt phố Kim Đồng, Phường Giáp Bát...",Cho Thuê,Nhà mặt tiền,NaN,8.0,13 phòng,None,None,https://cloud.muaban.net/images/thumb-detail/2...,916614768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,"Chính chủ cho thuê cả nhà khu Đầm Trấu, diện t...",15.000.000 đ/tháng,55 m²,"Đường Đầm Trấu, Phường Bạch Đằng, Quận Hai Bà ...",Chính chủ cho thuê cả nhà A11 số nhà 22 khu Đầ...,Cho Thuê,"Nhà hẻm, ngõ",NaN,4.0,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,915819577
251,"Cho thuê phòng trọ có điều hòa, tủ lạnh, bình ...",2.500.000 đ/tháng,20 m²,"Đường Nguyễn Chính, Phường Tân Mai, Quận Hoàng...","Cho thuê phòng trọ có điều hòa, tủ lạnh, bình ...",Cho Thuê,None,NaN,NaN,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,336072437
252,"nhà nguyên căn hẻm 5m,nhà mới xây, trệt, 2 lầu...",11.500.000 đ/tháng,"30 m² (3,0x10,0)","Đường Nghĩa Hòa, Phường 6, Quận Tân Bình, TP.HCM","Cho thuê nhà nguyên căn Phường 6, Quận Tân Bìn...",Cho Thuê,"Nhà hẻm, ngõ",NaN,NaN,2 phòng,3 WC,None,https://cloud.muaban.net/images/thumb-detail/2...,913626731
253,"Cho thuê phòng số 101 Cù Lao, Phường 2, Quận P...",4.000.000 đ/tháng,22 m²,"101, Đường Cù Lao, Phường 2, Quận Phú Nhuận, T...","Cho thuê 1 phòng , số 101 Cù Lao, Phường 2, Ph...",Cho Thuê,None,NaN,NaN,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,906360207


In [3]:
title = []                  # 1
price = []                  # 2
area = []                   # 3
bedroom = []        # 4
description = []            # 5
width = []                  # 6
direction = []              # 7
toilet = []         # 8
address = []               # 9
geo_code = []               # 10
law = []
floor = []
district = []
province_city = []
type = []
type_estate = []
image_links = []
phonenumber = []

In [4]:
import requests
import re
import pandas as pd

rentdata_df["geo_code"] = [None for _ in range(len(rentdata_df))]

def get_geocode(address):
    api_key = 'b48b7a0d92434d7ba9e13d0feb2db0ad'
    url = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}"

    response = requests.get(url)
    data = response.json()

    if data['results']:
        latitude = data['results'][0]['geometry']['lat']
        longitude = data['results'][0]['geometry']['lng']
        print(f"Latitude: {latitude}, Longitude: {longitude}")
        return [latitude, longitude]
    else:
        # Check if address contains "Đường"
        if "Đường" in address:
            # Remove "Đường" and retry
            modified_address = address.replace("Đường", "Đ.").strip()
            print(f"Retry with modified address: {modified_address}")
            return get_geocode(modified_address)
        else:
            print("No results found.")
            return None

def data_normalization(df):
    for i, address in enumerate(df["address"]):
        df.at[i, "geo_code"] = get_geocode(address)
        district, province_city = extract_district_and_province(address)
        if district == 'Thủ Đức -  Thủ Đức':
            district = 'Thủ Đức'
            
        df.at[i, "district"] = district.replace('Huyện', '').replace('Thành phố ','').replace('Quận', '').replace('TP.', '').replace(' -  2','').strip()
        if province_city == 'TP.HCM':
            province_city = 'Hồ Chí Minh'
            
        df.at[i, "province_city"] = province_city
        
    for i, price in enumerate(df['price']):
        # Loại bỏ các ký tự thừa và chỉ lấy phần số nguyên
        clean_price = price.replace('.', '').replace(' đ/tháng', '').strip()
        try:
            numeric_price = int(clean_price) / 1000000
            df.at[i, 'price'] = numeric_price
        except ValueError:
            df.at[i, 'price'] = None  # Hoặc thực hiện xử lý thay thế khác tùy theo yêu cầu
                
    for i, toilet in enumerate(df['toilet']):
        if isinstance(toilet, str):
            df.at[i, 'toilet'] = toilet.replace(' WC', '').strip()
        
    for i, bedroom in enumerate(df['bedroom']):        
        if isinstance(bedroom, str):
            df.at[i, 'bedroom'] = bedroom.replace(' phòng', '').strip()
    
    for i, area in enumerate(df['area']):
        if area is not None:
            # Split the area string to extract the width
            parts = area.split('(')
            if len(parts) > 1:
                width_part = parts[1].split('x')[0].strip().replace(',', '.')  # Extract and clean the width value
                try:
                    width_value = float(width_part)  # Convert to float if possible
                except ValueError:
                    width_value = 0.0  # Set default value in case of conversion error
                df.at[i, "width"] = width_value
            else:
                df.at[i, "width"] = None  # Set None if no width information found in the format

            # Update 'area' to keep only the first part (e.g., "42 m²")
            df.at[i, "area"] = " ".join(parts[0].split()[:1])            
        else:
            df.at[i, "width"] = None  # Set width to None if area is None
            
    for i, area in enumerate(df['area']):
        df.at[i, 'area'] = area.replace(' ,', '.').strip()
    
    for i, phonenumber in enumerate(df['phonenumber']):        
        if isinstance(phonenumber, str):
            phonenumber = phonenumber.replace(' ', '').replace('.', '').strip()
            # Kiểm tra nếu số đầu tiên khác 0 thì thêm số 0 vào đầu
            if phonenumber[0] != '0':
                phonenumber = '0' + phonenumber
            df.at[i, 'phonenumber'] = phonenumber.strip()
            
    for i, title in enumerate(df['title']):
        if isinstance(title, str):
            title = re.sub(r'[^\w\s,.()\–]', '', title)
            title = title.lower()
            title = title.capitalize()
            df.at[i, 'title'] = title.strip()    
        
def extract_district_and_province(address):
    parts = address.split(',')
    if len(parts) >= 2:
        district = parts[-2].strip()
        province_city = parts[-1].strip()
        return district, province_city
    return None, None

data_normalization(rentdata_df)

rentdata_df['type_estate'] = rentdata_df['type_estate'].replace({
    'Nhà mặt tiền': 'Nhà Mặt Phố',
    'Biệt thự, Villa': 'Nhà Biệt Thự, Liền Kề',
    'Nhà hẻm, ngõ': 'Nhà Trong Ngõ',
    'Chung cư': 'Căn Hộ Chung Cư',
    'Penthouse': 'Căn Hộ Chung Cư',
    'Tập thể, cư xá': 'Căn Hộ Chung Cư',
    'Căn hộ dịch vu, mini': 'Căn Hộ Chung Cư',
    'Đất thổ cư': 'Nhà Đất Thổ Cư',
    'Đất dự án, Khu dân cư': 'Đất Nền Dự Án',
    'Đất nông nghiệp, kho bãi': 'Đất Nông Nghiệp, Đất Vườn',
    'Chung cư': 'Căn Hộ Chung Cư',
    'Văn phòng': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'Cửa hàng, shophouse': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'Mặt bằng kinh doanh': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'Nhà xưởng, nhà kho': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'Bãi để xe': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'Đất trống': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
})

def write_data(df, field, data_list):
    if not isinstance(data_list, list):
        data_list = []  # Initialize as empty list if not already
    
    if field in df.columns:
        for value in df[field]:
            data_list.append(value)
    else:
        print(f"Error: '{field}' column not found in the DataFrame.")
 
# Populate lists with data from DataFrame
write_data(rentdata_df, "title", title)
write_data(rentdata_df, "price", price)
write_data(rentdata_df, "area", area)
write_data(rentdata_df, "type", type)
write_data(rentdata_df, "type_estate", type_estate)
write_data(rentdata_df, "image_links", image_links)
write_data(rentdata_df, "bedroom", bedroom)
write_data(rentdata_df, "description", description)
write_data(rentdata_df, "width", width)
write_data(rentdata_df, "direction", direction)
write_data(rentdata_df, "toilet", toilet)
write_data(rentdata_df, "address", address)
write_data(rentdata_df, "law", law)
write_data(rentdata_df, "floor", floor)
write_data(rentdata_df, "geo_code", geo_code)
write_data(rentdata_df, "district", district)
write_data(rentdata_df, "province_city", province_city)
write_data(rentdata_df, "phonenumber", phonenumber)

# Display the updated DataFrame
print("\nUpdated DataFrame:")


Latitude: 21.03587, Longitude: 105.82163
Latitude: 10.82302, Longitude: 106.62965
Latitude: 10.8078298, Longitude: 106.709329
Latitude: 21.0282433, Longitude: 105.8415359
Retry with modified address: Đ. Kim Đồng, Phường Giáp Bát, Quận Hoàng Mai, Hà Nội
Latitude: 20.98359, Longitude: 105.841149
Latitude: 21.02813, Longitude: 105.85358
Latitude: 10.8063236, Longitude: 106.720263
Latitude: 10.7074107, Longitude: 106.581652
Latitude: 21.0287928, Longitude: 105.8429216
Retry with modified address: 49D, Đ. 18, Phường Linh Trung, TP. Thủ Đức - Quận Thủ Đức, TP.HCM
No results found.
Latitude: 20.9916184, Longitude: 105.86303
Latitude: 21.0031848, Longitude: 105.8202308
Latitude: 10.78069, Longitude: 106.69944
Latitude: 16.0658435, Longitude: 108.1950278
Latitude: 10.8675368, Longitude: 106.6263371
Latitude: 10.6756697, Longitude: 106.7628069
Latitude: 16.0704792, Longitude: 108.2212181
Latitude: 21.0469979, Longitude: 105.8784092
Latitude: 10.8282636, Longitude: 106.6202489
Latitude: 21.051961

In [5]:
rentdata_df

,title,price,area,address,description,type,type_estate,law,floor,bedroom,toilet,direction,image_links,phonenumber,geo_code,district,province_city,width
0,"Cho thuê căn hộ tầng 3, 4, 5 trong toà nhà 8 t...",8.7,65,"Đường Giang Văn Minh, Phường Đội Cấn, Quận Ba ...","Nhà mình còn 3 căn hộ tầng 3, 4, 5 trong toà n...",Cho Thuê,Căn Hộ Chung Cư,NaN,NaN,1,1,None,https://cloud.muaban.net/images/thumb-detail/2...,913236393,"[21.03587, 105.82163]",Ba Đình,Hà Nội,None
1,Cho thuê mặt bằng tầng trệt đường số 6 khu cit...,16.0,100,"Cityland Park Hills, Đường số 6, Phường 10, Qu...",Cho thuê mặt bằng tầng trệt đường số 6 khu Cit...,Cho Thuê,"Văn phòng, TTTM, Cửa hàng, Kiot",NaN,NaN,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,903101485,"[10.82302, 106.62965]",Gò Vấp,Hồ Chí Minh,None
2,Cho thuê vp giá rẻ nhất bình thạnh,12.0,56,"110, Đường Đinh Bộ Lĩnh, Phường 26, Quận Bình ...",Cơ hội vàng cho doanh nghiệp: Văn phòng đẳng c...,Cho Thuê,"Văn phòng, TTTM, Cửa hàng, Kiot",NaN,NaN,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,983414444,"[10.8078298, 106.709329]",Bình Thạnh,Hồ Chí Minh,None
3,"Cho thuê cửa hàng phố lê duẩn, nhìn sang điện ...",0.123456,60,"Đường Lê Duẩn, Quận Ba Đình, Hà Nội","Cần cho thuê cửa hàng phố Lê Duẩn, nhìn sang Đ...",Cho Thuê,"Văn phòng, TTTM, Cửa hàng, Kiot",NaN,NaN,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,934188668,"[21.0282433, 105.8415359]",Ba Đình,Hà Nội,None
4,"Chính chủ cho thuê nhà mặt phố kim đồng, phườn...",30.0,60,"Đường Kim Đồng, Phường Giáp Bát, Quận Hoàng Ma...","Cho Thuê Nhà mặt phố Kim Đồng, Phường Giáp Bát...",Cho Thuê,Nhà Mặt Phố,NaN,8.0,13,None,None,https://cloud.muaban.net/images/thumb-detail/2...,916614768,"[20.98359, 105.841149]",Hoàng Mai,Hà Nội,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,"Chính chủ cho thuê cả nhà khu đầm trấu, diện t...",15.0,55,"Đường Đầm Trấu, Phường Bạch Đằng, Quận Hai Bà ...",Chính chủ cho thuê cả nhà A11 số nhà 22 khu Đầ...,Cho Thuê,Nhà Trong Ngõ,NaN,4.0,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,915819577,"[21.00626, 105.85537]",Hai Bà Trưng,Hà Nội,None
251,"Cho thuê phòng trọ có điều hòa, tủ lạnh, bình ...",2.5,20,"Đường Nguyễn Chính, Phường Tân Mai, Quận Hoàng...","Cho thuê phòng trọ có điều hòa, tủ lạnh, bình ...",Cho Thuê,None,NaN,NaN,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,336072437,"[20.9836387, 105.8477359]",Hoàng Mai,Hà Nội,None
252,"Nhà nguyên căn hẻm 5m,nhà mới xây, trệt, 2 lầu...",11.5,30,"Đường Nghĩa Hòa, Phường 6, Quận Tân Bình, TP.HCM","Cho thuê nhà nguyên căn Phường 6, Quận Tân Bìn...",Cho Thuê,Nhà Trong Ngõ,NaN,NaN,2,3,None,https://cloud.muaban.net/images/thumb-detail/2...,913626731,"[10.784986, 106.6593731]",Tân Bình,Hồ Chí Minh,3.0
253,"Cho thuê phòng số 101 cù lao, phường 2, quận p...",4.0,22,"101, Đường Cù Lao, Phường 2, Quận Phú Nhuận, T...","Cho thuê 1 phòng , số 101 Cù Lao, Phường 2, Ph...",Cho Thuê,None,NaN,NaN,None,None,None,https://cloud.muaban.net/images/thumb-detail/2...,906360207,"[10.82302, 106.62965]",Phú Nhuận,Hồ Chí Minh,None


In [6]:
rentdata_df.to_csv('rent_data.csv', encoding='utf-8', index=False)

In [7]:
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with connection.cursor() as cursor:
        # Xóa dữ liệu từ cả hai bảng pools_bds và pools_image
        cursor.execute("""
            DELETE pb, pi
            FROM pools_bds pb
            JOIN pools_image pi ON pb.BDS_id = pi.BDS_id
            WHERE pb.kind = 'Cho Thuê' AND pi.content LIKE '%muaban.net%'
        """)
        connection.commit()

finally:
    connection.close()


In [ ]:
import csv
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL với mã hóa utf8mb4
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

def is_decimal(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

try:
    with connection.cursor() as cursor:
        
        # Đọc dữ liệu từ file CSV và nhập vào MySQL
        with open('rent_data.csv', 'r', encoding='utf-8') as file:
            csv_data = csv.DictReader(file)
            for row in csv_data:
                try:
                    if not row['type_estate']:
                        continue
                    
                    district = row['district']
                    province_city = row['province_city']
                    
                    # Kiểm tra tồn tại address
                    cursor.execute("SELECT address_id FROM pools_address WHERE district = %s AND province_city = %s", (district, province_city))
                    existing_address = cursor.fetchone()
                    if existing_address:
                        address_id = existing_address['address_id']
                    else:
                        # Nếu không tìm thấy địa chỉ, bỏ qua dòng này
                        print(f"Không tìm thấy địa chỉ cho dòng: {row}")
                        continue
                    
                    # Kiểm tra pools_typeofbds 
                    bds_type = row['type_estate']
                    cursor.execute("SELECT Type_Id FROM pools_typeofbds WHERE TypeName = %s", (bds_type,))
                    existing_type = cursor.fetchone()
                    if existing_type:
                        type_id = existing_type['Type_Id']
                    else:
                        # Nếu không tìm thấy loại bất động sản, bỏ qua dòng này
                        print(f"Không tìm thấy loại bất động sản cho dòng: {row}")
                        continue
                    
                    # Thêm dữ liệu vào bảng pools_bds
                    title = row['title']
                    price = row['price']
                    area = row['area']
                    kind = row['type']
                    law = row['law']
                    bedroom = row['bedroom']
                    floor = row['floor']
                    description = row['description']
                    width = row['width']
                    direction = row['direction']
                    toilet = row['toilet']
                    location = row['address']
                    geocode = row['geo_code']
                    phonenumber = row['phonenumber']
                    
                    # Handle 'price' column
                    if price.strip().lower() == 'thỏa thuận':
                        price = 0  # or set to a default value, like 0
                    elif not is_decimal(price):
                        price = 0  # or set to a default value

                    # Handle 'width' column
                    if not is_decimal(width):
                        width = 0  # or set to a default value

                    # Handle 'bedroom' column
                    if not bedroom.strip() or bedroom.lower() == 'null':
                        bedroom = 0  # or set to a default value
                       
                    # Handle 'floor' column 
                    if not floor.strip() or floor.lower() == 'null':
                        floor = 0  # or set to a default value

                    # Handle 'toilet' column (if needed)
                    if not is_decimal(toilet):
                        toilet = 0  # or set to a default value

                    # Kiểm tra geocode nếu là None hoặc rỗng, bỏ qua dòng này
                    if not geocode:
                        # print(f"Không có giá trị geocode cho dòng: {row}")
                        continue

                    # Chuyển geo_code thành string nếu nó là list
                    if isinstance(geocode, list):
                        geocode = str(geocode)

                    # Thêm dữ liệu vào bảng pools_bds
                    cursor.execute("""
                        INSERT INTO pools_bds (title, price, area, kind, law, bedroom, floor, phonenumber, address_id, Type_id, description, width, direction, toilet, location, geocode)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (title, price, area, kind, law, bedroom, floor, phonenumber, address_id, type_id, description, width, direction, toilet, location, geocode))
                    connection.commit()
                    
                    # Lấy BDS_id của BDS vừa thêm
                    cursor.execute("SELECT LAST_INSERT_ID()")
                    bds_id = cursor.fetchone()['LAST_INSERT_ID()']
                    
                    # Thêm dữ liệu vào bảng pools_image (nếu có)
                    if 'image_links' in row and row['image_links']:  # Kiểm tra xem cột 'image_link' có tồn tại và có giá trị không
                        image_link = row['image_links']
                        cursor.execute("INSERT INTO pools_image (BDS_id, content) VALUES (%s, %s)", (bds_id, image_link))
                        connection.commit()
                        
                except Exception as e:
                    print(f"Lỗi khi xử lý dòng: {row}")
                    print(e)
finally:
    connection.close()
